# **Bug Localization - DNN**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
project_path = 'drive/MyDrive/Colab Notebooks/BugLocalization'

import sys
sys.path.append(project_path)

## 1. Tiền xử lý dữ liệu

In [3]:
!pip install pickle
!pip install inflection
!pip install nltk
!pip install javalang

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


In [4]:
from preprocessing import Parser, ReportPreprocessing, SrcPreprocessing
from projectDatasets import aspectj, eclipse, swt, tomcat, birt
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


True

In [5]:
import os
import pickle

# Đường dẫn thư mục models trong dự án
models_dir = f"{project_path}/models"
os.makedirs(models_dir, exist_ok=True)

bug_reports_path = f"{models_dir}/preprocessed_bug_reports.pkl"
src_reports_path = f"{models_dir}/preprocessed_src_reports.pkl"

with open(src_reports_path, "rb") as f:
    aspectj_src_files = pickle.load(f)
    eclipse_src_files = pickle.load(f)
    swt_src_files = pickle.load(f)
    tomcat_src_files = pickle.load(f)
    birt_src_files = pickle.load(f)


with open(bug_reports_path, "rb") as f:
    aspectj_bug_reports = pickle.load(f)
    eclipse_bug_reports = pickle.load(f)
    swt_bug_reports = pickle.load(f)
    tomcat_bug_reports = pickle.load(f)
    birt_bug_reports = pickle.load(f)

print("Data load successfully!")

Data load successfully!


## 2. Chia fold, gán nhãn dữ liệu

### Chia fold cho bug reports
- 3 fold cho Aspectj
- 10 fold cho các projects còn lại
- Sắp xếp bug reports rồi chia theo thời gian từ cũ đến mới (fold i để test và fold i + 1 để train)

In [6]:
# Sắp xếp lại bug reports theo thời gian từ cũ đến mới
sorted_aspectj_bug_reports = sorted(aspectj_bug_reports.items(), key=lambda x: x[1].report_time)
sorted_eclipse_bug_reports = sorted(eclipse_bug_reports.items(), key=lambda x: x[1].report_time)
sorted_swt_bug_reports = sorted(swt_bug_reports.items(), key=lambda x: x[1].report_time)
sorted_tomcat_bug_reports = sorted(tomcat_bug_reports.items(), key=lambda x: x[1].report_time)
sorted_birt_bug_reports = sorted(birt_bug_reports.items(), key=lambda x: x[1].report_time)

In [198]:
# Lấy một báo cáo lỗi cụ thể, ví dụ báo cáo lỗi đầu tiên trong danh sách
bug_report = sorted_aspectj_bug_reports[0]
bug_report2 = sorted_aspectj_bug_reports[1]

# Truy cập vào report_time của bug report
report_time = bug_report[1].report_time  # bug_report[1] là đối tượng BugReport
report_time2 = bug_report2[1].report_time

# In ra thời gian báo cáo, kiểm tra xem đúng thứ tự từ cũ đến mới chưa
print(report_time)
print(report_time2)

print(len(sorted_aspectj_bug_reports))

2002-03-13 12:40:54
2002-12-30 16:40:03
593


In [8]:
import numpy as np
# Chia fold theo thứ tự thời gian
def split_time_ordered_folds(sorted_data, n_folds):
    fold_size = len(sorted_data) // n_folds # Tính xấp xỉ fold_size cần
    folds = []

    # Tạo các indices cho từng fold
    indices = np.arange(len(sorted_data)) # Tạo mảng chỉ số cho các phần tử reports
    fold_indices = np.array_split(indices, n_folds) # Chia mảng indices thành n folds, kích thước bằng nhau nhất có thể

    # Lưu các fold và indices của chúng
    fold_data = []
    for fold_idx in fold_indices:
        fold_data.append([sorted_data[i] for i in fold_idx])
        # Thêm data vào fold data nếu chỉ số có ở trong tập fold_indices

    return fold_data

# Chia thành các fold theo yêu cầu

# AspectJ chia 3 fold
aspectj_folds = split_time_ordered_folds(sorted_aspectj_bug_reports, 3)
 # Các project khác chia 10 fold
eclipse_folds = split_time_ordered_folds(sorted_eclipse_bug_reports, 10)
swt_folds = split_time_ordered_folds(sorted_swt_bug_reports, 10)
tomcat_folds = split_time_ordered_folds(sorted_tomcat_bug_reports, 10)
birt_folds = split_time_ordered_folds(sorted_birt_bug_reports, 10)

### Gán nhãn dữ liệu
- Nếu liên quan --> label là 1
- Nếu không liên quan --> label là 0
- Vấn đề: label 0 sẽ có rất nhiều, trong khi label 1 rất ít (skewed distribution --> cách giải quyết: data-sample manipulation, cost-sensitive learning(focal-loss function))
  - Giải quyết bài toán: Chọn ra top k các lớp 0 có similarity lớn, khó phân biệt với lớp 1 để train(hard negative sampling)

Hàm tính cosine similarity giữa 2 văn bản (lexical similarity)

In [9]:
def bug_report_to_text(bug_report):
    text_parts = []

    # Kiểm tra và thêm phần tóm tắt đã gắn nhãn từ bug_report
    if bug_report.pos_tagged_summary is not None and 'unstemmed' in bug_report.pos_tagged_summary:
        text_parts.append(" ".join(bug_report.pos_tagged_summary['unstemmed']))

    # Kiểm tra và thêm phần mô tả đã gắn nhãn từ bug_report
    if bug_report.pos_tagged_description is not None and 'unstemmed' in bug_report.pos_tagged_description:
        text_parts.append(" ".join(bug_report.pos_tagged_description['unstemmed']))

    # Kết hợp tất cả các phần và trả về kết quả
    text = " ".join(text_parts)
    return text

def src_report_to_text(src_report):
    text_parts = []

    # Kiểm tra và thêm phần bình luận đã gắn nhãn từ source_report
    if src_report.pos_tagged_comments is not None and 'unstemmed' in src_report.pos_tagged_comments:
        text_parts.append(" ".join(src_report.pos_tagged_comments['unstemmed']))

    # Kiểm tra và thêm tên lớp từ source_report
    if src_report.class_names is not None:
        text_parts.append(" ".join(src_report.class_names))

    # Kiểm tra và thêm các thuộc tính từ source_report
    if src_report.attributes is not None:
        text_parts.append(" ".join(src_report.attributes))

    # Kiểm tra và thêm tên phương thức từ source_report
    if src_report.method_names is not None:
        text_parts.append(" ".join(src_report.method_names))

    # Kiểm tra và thêm biến từ source_report
    if src_report.variables is not None:
        text_parts.append(" ".join(src_report.variables))

    # Kết hợp tất cả các phần và trả về kết quả
    text = " ".join(text_parts)
    return text

In [10]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Đầu vào: 2 text (bug report, src_file)
def lexical_similarity(bug_report, src_report):
    # Chuyển bug_report và src_report thành văn bản
    bug_report_text = bug_report_to_text(bug_report)
    src_report_text = src_report_to_text(src_report)

    # Kết hợp văn bản của bug_report và src_report
    all_texts = [bug_report_text, src_report_text]

    # Tính TF-IDF cho văn bản
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_texts)

    # Lấy các vector TF-IDF cho bug_report và src_report
    bug_report_tfidf = tfidf_matrix[0]  # Vector của bug_report
    src_report_tfidf = tfidf_matrix[1]  # Vector của src_report

    # Tính cosine similarity giữa bug_report và src_report
    similarity = cosine_similarity(bug_report_tfidf, src_report_tfidf)

    return similarity[0][0]

Lấy ra tập dương tính

In [11]:
# Đầu vào: 1 bug report, tập source files của project đó
# Đầu ra: các source files liên quan đến bug report đó

def src_positive(bug_report, src_files):
  src_pos = []
  bug_report_fixed_files = list(
    f.strip() for f in bug_report.fixed_files if f.strip() != '.' and f.strip() != ''
)
  for src_address, src_file in src_files.items():
    if src_address in bug_report_fixed_files:
      src_pos.append(src_file)
  return src_pos #[src_file, src_file,....]

Lấy ra tập âm tính


In [12]:
# Đầu vào: 1 bug report, tập source files của project đó
# Đầu ra: top 500 file không liên quan đến bug report nhưng có lexical similarity cao nhất

def src_negative(bug_report, src_files, k=500):
  src_neg = []
  bug_report_fixed_files = list(
    f.strip() for f in bug_report.fixed_files if f.strip() != '.' and f.strip() != ''
  )
  sim_scores = []
  for src_address , src_file in src_files.items():
    if src_address not in bug_report_fixed_files: # Kiểm tra không ở trong tập positive
      sim_score = lexical_similarity(bug_report, src_file) # Tính cosine similarity giữa 2 file
      sim_scores.append((src_file, sim_score))
  sim_scores.sort(key=lambda x: x[1], reverse=True) # Sắp xếp theo phần tử thứ 2 (sim_score), reverse=True --> giảm dần theo sim_score
  # Chọn ra top k src file âm tính có sim_score cao nhất
  for i in range(k):
    src_neg.append(sim_scores[i][0])
  return src_neg # [src_file, src_file,...]

Label data

In [13]:
print(aspectj_folds[0][0][1].fixed_files)

['tests/src/org/aspectj/systemtest/ajc150/Ajc150Tests.java', ' weaver/src/org/aspectj/weaver/reflect/ReflectionBasedReferenceTypeDelegateFactory.java', '.']


In [14]:
# Đầu vào: 1 fold bug_reports dùng để train, src_files của project đó
# Đầu ra: 1 list các tuple (bug_report, src_file, label)

# 1 fold bug_reports có dạng [(bug_id, BugReport Object),....]

def label_data_train(prj_bug_reports_fold, src_files):
  data_train = []
  for bug_id, bug_report in prj_bug_reports_fold:
    src_pos = src_positive(bug_report, src_files)
    src_neg = src_negative(bug_report, src_files)
    for src_file in src_pos:
      data_train.append((bug_report, src_file, 1))
    for src_file in src_neg:
      data_train.append((bug_report, src_file, 0))
  return data_train, src_pos, src_neg

In [15]:
# Đầu vào: 1 fold bug_reports dùng để test, src_files của project đó
# Đầu ra: 1 list các tuple (bug_report, src_file, label)

def label_data_test(prj_bug_reports_fold, src_files):
  data_test = []
  for bug_id, bug_report in prj_bug_reports_fold:
    bug_report_fixed_files = list(
    f.strip() for f in bug_report.fixed_files if f.strip() != '.' and f.strip() != ''
    )
    for src_address, src_file in src_files.items():
      if src_address in bug_report_fixed_files:
        data_test.append((bug_report, src_file, 1))
      else:
        data_test.append((bug_report, src_file, 0))
  return data_test

### Label và lưu lại dữ liệu train, test

In [30]:
def save_fold_data(project_name, project_src_files, project_folds):
    project_dir = f"{models_dir}/{project_name}"
    print(f"Saving project: {project_name}")
    os.makedirs(project_dir, exist_ok=True)

    n_folds = len(project_folds)

    for i in range(n_folds - 1):  # fold i dùng để train, fold i+1 để test
        # Gán dữ liệu train/test
        data_train, src_pos, src_neg = label_data_train(project_folds[i], project_src_files) #Train fold i
        data_test = label_data_test(project_folds[i + 1], project_src_files) # Test fold i + 1

        # Lưu bằng pickle
        with open(os.path.join(project_dir, f"fold_{i}_train.pkl"), "wb") as f:
            pickle.dump(data_train, f)

        with open(os.path.join(project_dir, f"fold_{i}_test.pkl"), "wb") as f:
            pickle.dump(data_test, f)

        print(f"Saved fold {i}: train={len(data_train)}, test={len(data_test)}")

In [ ]:
# save_fold_data ('aspectj', aspectj_src_files, aspectj_folds)
save_fold_data ('eclipse', eclipse_src_files, eclipse_folds)
save_fold_data ('swt', swt_src_files, swt_folds)
save_fold_data ('tomcat', tomcat_src_files, tomcat_folds)
save_fold_data('birt', birt_src_files, birt_folds)

In [17]:
def load_fold_data(project_name, fold_idx):
    project_dir = f"{models_dir}/{project_name}"

    train_file = os.path.join(project_dir, f"fold_{fold_idx}_train.pkl")
    test_file = os.path.join(project_dir, f"fold_{fold_idx}_test.pkl")
    src_pos_file = os.path.join(project_dir, f"fold_{fold_idx}_src_pos.pkl")
    src_neg_file = os.path.join(project_dir, f"fold_{fold_idx}_src_neg.pkl")

    # Load tập train, tập test
    with open(train_file, "rb") as f:
        data_train = pickle.load(f)
    with open(test_file, "rb") as f:
        data_test = pickle.load(f)
    return data_train, data_test

In [174]:
# Load fold 0 của project aspectj để kiểm tra
data_train, data_test  = load_fold_data('aspectj', 0)
print(f"Train data size: {len(data_train)}, Test data size: {len(data_test)}")

Train data size: 99523, Test data size: 929808


In [175]:
count = 0
count_label_1 = 0
for report, file, label in data_train:
  count += 1
  if label == 1:
    count_label_1 += 1
    #print(f"{report}, {file}, {label}")
print(f"Label 1: {count_label_1}, Label 0: {count - count_label_1}")

Label 1: 523, Label 0: 99000


In [53]:
#Lấy ra 1 phần của test set aspectj
import random
def stratified(data_test, sample_size, min_label_1 = 1):
    data_label_1 = [d for d in data_test if d[2] == 1]
    data_label_0 = [d for d in data_test if d[2] == 0]

    total = len(data_test)
    ratio_label_1 = len(data_label_1) / total

    # Tính số mẫu cần lấy
    num_label_1 = max(int(sample_size * ratio_label_1), min_label_1)
    num_label_1 = min(num_label_1, len(data_label_1))  # không vượt quá số thực tế
    num_label_0 = sample_size - num_label_1

    sampled_label_1 = random.sample(data_label_1, num_label_1)
    sampled_label_0 = random.sample(data_label_0, num_label_0)

    sampled_data = sampled_label_1 + sampled_label_0
    random.shuffle(sampled_data)
    return sampled_data

## 3. Trích xuất đặc trưng

### 3.1. Lexical similarity

- Hàm bug2text và src2text ở trên

In [34]:
# Đầu vào: 1 pair trong tập data train (1 tuple (bug_report, source_file, label))
# Đầu ra: lexical similarity của pair đó

def get_lexical_similarity(pair):
    similarity = lexical_similarity(pair[0], pair[1])
    return similarity

In [35]:
get_lexical_similarity(data_train[0])

np.float64(0.24505498492568153)

### 3.2. Semantic similarity
Do có lexical mismatch khi bug reports sử dụng ngôn ngữ tự nhiên, còn source code lại sử dụng ngôn ngữ lập trình --> Tìm hiểu thêm về semantic relationship giữa các từ phụ thuộc vào context nó xuất hiện

- Sử dụng pretrained Glove để chuyển từ --> vectors
  - Glove: word embeddings, dựa trên co-occurrence matrix của các từ trong văn bản. Glove sẽ tìm 1 bộ các vector sao cho tần suất đồng xuất hiện của các từ trong không gian vector gần với tần suất đồng xuất hiện trong co-occurence matrix

In [36]:
# Load pretrained glove
glove_path = '/content/drive/MyDrive/Colab Notebooks/BugLocalization/glove.6B/glove.6B.100d.txt'

def load_glove_model(glove_file):
    print("Loading GloVe model...")
    model = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.split()
            word = parts[0]
            vector = np.asarray(parts[1:], dtype='float32')
            model[word] = vector
    print(f"Model loaded with {len(model)} words.")
    return model

glove_model = load_glove_model(glove_path)

Loading GloVe model...
Model loaded with 400000 words.


In [37]:
import torch
import torch.nn.functional as F

def embed_text_tensor(text, glove_dict, tfidf_weights, vocab, dim=100):
    tokens = text.split()
    vecs = []
    weights = []

    for token in tokens:
        if token in glove_dict and token in vocab:
            vecs.append(torch.tensor(glove_dict[token]))
            weights.append(tfidf_weights.get(token, 0))  # Lấy TF-IDF weight nếu có, nếu không thì 0

    if not vecs:
        return torch.zeros(dim)

    vecs = torch.stack(vecs)  # (n_tokens, dim)
    weights = torch.tensor(weights).unsqueeze(1)  # (n_tokens, 1)

    # Nhân từng vector với trọng số TF-IDF
    weighted_vecs = vecs * weights

    # Tính tổng và chuẩn hóa bằng cách chia cho tổng trọng số
    return weighted_vecs.sum(dim=0) / weights.sum()

In [38]:
def get_semantic_similarity(pair,glove_dict, dim=100, device="cpu"):
    # Chuyển bug_report và src_report thành văn bản
    bug_report = pair[0]
    src_report= pair[1]
    bug_report_text = bug_report_to_text(bug_report)  # 1 bug report
    src_report_text = src_report_to_text(src_report)  # 1 source report

    # Fit TF-IDF để lấy idf weight chỉ cho cặp bug_report và src_report
    tfidf = TfidfVectorizer()
    tfidf.fit([bug_report_text, src_report_text])
    vocab = tfidf.vocabulary_
    idf_weights = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))

    # Embed text thành tensor
    def embed_text_tensor_with_tfidf(text):
        return embed_text_tensor(text, glove_dict, idf_weights, vocab, dim)

    # Embed bug_report và src_report thành vectors
    bug_vec = embed_text_tensor_with_tfidf(bug_report_text).to(device)
    src_vec = embed_text_tensor_with_tfidf(src_report_text).to(device)

    # Tính cosine similarity giữa bug_report và src_report
    similarity = cosine_similarity([bug_vec.cpu().numpy()], [src_vec.cpu().numpy()])

    return similarity[0][0]  # Trả về giá trị similarity duy nhất

In [39]:
get_semantic_similarity(data_train[0], glove_model)

np.float64(0.860221418588926)

### 3.3. Similar Bug Reports  
Dựa trên các bug reports trước đó mà source file đã được fix, xem xem có giống các bug report đã từng sửa cùng file đó không

In [40]:
def find_source_files_for_bug_report(target_bug_report, dataset):
    # Tìm tất cả các SourceFile liên quan đến BugReport
    source_files = [source_file for bug_report, source_file, label in dataset if bug_report == target_bug_report]
    return source_files

In [41]:
test_src_files = find_source_files_for_bug_report(data_train[2001][0], data_train)

In [42]:
def find_bug_reports_fix_source_files(target_source_files, dataset):
    # Tìm tất cả BugReport đã fix SourceFiles (Bug Report đã sửa file đó)(label = 1)
    bug_reports = [bug_report for bug_report, source_file, label in dataset if source_file in target_source_files and label == 1]
    return bug_reports

In [43]:
print(len(find_bug_reports_fix_source_files(test_src_files, data_train)))

193


In [44]:
def get_previous_bug_reports(bug_report, dataset):
  source_files = find_source_files_for_bug_report(bug_report, dataset)
  bug_reports = find_bug_reports_fix_source_files(source_files, dataset)
  return bug_reports

In [45]:
def get_similar_bugs_score(pair, dataset):
  #pair (BugReport, SrcFile, Label)
  previous_bug_reports = get_previous_bug_reports(pair[0], dataset)
  if len(previous_bug_reports) == 0:
    return 0
  else:
    sum = 0
    for bug_report in previous_bug_reports:
      bug_report_text = bug_report_to_text(pair[0])
      bug2_report_text = bug_report_to_text(bug_report)
      # Kết hợp văn bản của bug_report và bug2_report
      all_texts = [bug_report_text, bug2_report_text]

      # Tính TF-IDF cho văn bản
      vectorizer = TfidfVectorizer()
      tfidf_matrix = vectorizer.fit_transform(all_texts)

      # Lấy các vector TF-IDF cho bug_report và bug2_report
      bug_report_tfidf = tfidf_matrix[0]  # Vector của bug_report
      bug2_report_tfidf = tfidf_matrix[1]  # Vector của bug2_report

      # Tính cosine similarity giữa bug_report và bug2_report
      similarity = cosine_similarity(bug_report_tfidf, bug2_report_tfidf)
      sum += similarity[0][0]

    return sum / len(previous_bug_reports)

In [46]:
get_similar_bugs_score(data_train[2001], data_train)

np.float64(0.04978423557773192)

### 3.4. Code change history
File mà đã fix rất lâu về trước hoặc chưa bao giờ fix thì sẽ ít khả năng lỗi hơn
- Trả về 1 / khoảng cách ngày tới lần fix gần nhất
 --> Nếu như khoảng cách càng xa, score càng bé

In [47]:
def get_code_change_history_score(pair, dataset):
  report_time = pair[0].report_time
  # Trả về tất cả các bug reports mà đã fix source file
  fixed_bug_reports_for_src_file = [bug_report for bug_report, source_file, label in dataset if source_file == pair[1] and label == 1]
  min_time = report_time
  # Lấy report time của các bug_reports đã fix xong, nếu nhỏ hơn report time thì gán min time vào
  for bug_report in fixed_bug_reports_for_src_file:
    if bug_report.report_time < report_time:
      min_time = min(min_time, bug_report.report_time)

  elapsed_days = (report_time - min_time).days
  if elapsed_days == 0:
    return 1 #Nếu trả về 1 thì tức là file đó chưa từng được fix lần nào trước report time của bug_report
  else:
    return 1 / elapsed_days

In [48]:
get_code_change_history_score(data_train[2001], data_train)

1

### 3.5. Bug fixing frequency
Số lần mà source file được sửa trước report time cũng sẽ ảnh hưởng đến xác suất file đó có lỗi hay không  
(Nếu như sửa càng nhiều --> xác suất lỗi càng cao)

In [49]:
def get_bug_fixing_freq_score(pair, dataset):
  report_time = pair[0].report_time
  # Trả về tất cả các bug reports mà đã fix source file
  fixed_bug_reports_for_src_file = [bug_report for bug_report, source_file, label in dataset
                                    if source_file == pair[1] and label == 1]
  # Tìm trong các bug_report đã fix nếu report time nhỏ hơn bug_report hiện tại (hay đã fix trước đó)
  count = 0
  for bug_report in fixed_bug_reports_for_src_file:
    if bug_report.report_time < report_time:
      count +=1

  return count

In [50]:
get_bug_fixing_freq_score(data_train[2001], data_train)

0

## 4. Tạo tập train và tập test

### Gộp các features đã tính
Gộp các features đã tính thành 1 vector `[lex_sim, sem_sim, prev_bug_sim, change_history, change_freq]`

In [51]:
def extract_features(pair, dataset, glove_dict):
    lex_sim = get_lexical_similarity(pair)
    sem_sim = get_semantic_similarity(pair, glove_dict)
    prev_bug_sim = get_similar_bugs_score(pair, dataset)
    change_history = get_code_change_history_score(pair, dataset)
    change_freq = get_bug_fixing_freq_score(pair, dataset)

    features_vector = np.array([lex_sim, sem_sim, prev_bug_sim, change_history, change_freq])
    return features_vector

In [52]:
extract_features(data_train[0], data_train, glove_model)

array([0.24505498, 0.86022142, 0.02084132, 1.        , 0.        ])

### Tạo minibatch cho dữ liệu train bằng Bootstraping



In [87]:
# Hàm dùng để tạo data: data có dạng [(vector, label),....]; dùng cho cả tập train và tập test
# Tập train: fold i; tập test: fold i + 1
def create_data(dataset, glove_dict):
  print(f"Creating data for {len(dataset)} samples")
  data = []
  for pair in dataset:  #pair: (bug_report, src_file, label) --> label: pair[2]
    x_pair = extract_features(pair, dataset, glove_dict)
    y_pair = pair[2]
    data.append((x_pair, y_pair))
  return data

In [63]:
# Lấy ra tập (vector, label) positive
def get_pos(data):
  pos_set = []
  for vector, label in data:
    if label == 1:
      pos_set.append((vector, 1))
  return pos_set

# Lấy ra tập (vector, label) negative
def get_neg(data):
  neg_set = []
  for vector, label in data:
    if label == 0:
      neg_set.append((vector, 0))
  return neg_set

In [99]:
import random
def create_minibatch(data, batch_size):
  print(f"Creating minibatch for {len(data)} samples")
  mini_batches = []
  pos_set = get_pos(data) #[(X_pos, 1)]
  neg_set = get_neg(data) #[(X_neg, 0)]

  n_pos = len(pos_set)
  n_neg = len(neg_set)
  K = n_neg // batch_size

  Sn = batch_size // 2
  Sp = batch_size - Sn

  #Chia negative thành K phần, từng phần có Sn samples
  neg_batches = [neg_set[i * Sn : (i + 1) * Sn] for i in range(K)]

  #Copy tập postive để rút ngẫu nhiên
  temp_pos = pos_set.copy()

  for i in range(K):
    if len(temp_pos) < Sp:
      temp_pos = pos_set.copy()

    random.shuffle(temp_pos)
    #pos_i = random.sample(temp_pos, Sp)
    pos_i = random.choices(temp_pos, k=Sp)

    #for item in pos_i:
    #  temp_pos.remove(item)



    #batch = neg_batches[i] + pos_i
    #random.shuffle(batch)
    batch_inputs = [x for x, y in neg_batches[i] + pos_i]
    batch_labels = [y for x, y in neg_batches[i] + pos_i]

    mini_batches.append((batch_inputs, batch_labels))
  return mini_batches # 1 batch[(X,y),...]; 1 minibatch: list các batch

## 5. Huấn luyện và đánh giá

### Định nghĩa mô hình

In [65]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Định nghĩa hàm focal loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'): #Cho kết quả tổng quát trên toàn bộ batch
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.BCELoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)  # pt là xác suất của nhãn đúng
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss


# Mô hình DNN
class BugLocalization(nn.Module):
    def __init__(self, input_dim):
        super(BugLocalization, self).__init__()
        # Các lớp ẩn với kích thước 300 và 150 node
        self.fc1 = nn.Linear(input_dim, 300)
        self.fc2 = nn.Linear(300, 150)
        self.fc3 = nn.Linear(150, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc3(x))
        return x

### Huấn luyện mô hình

In [66]:
batch_size = 64
epochs = 30

In [67]:
def train_model(model, mini_batches, criterion, optimizer, device, project_name, epochs):
    model.train()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    save_dir = f"{models_dir}/{project_name}_training_model"
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(epochs):
        running_loss = 0.0
        batch_count = 0
        # 1 mini_batch: 1 list các batch
        # 1 batch: (batch_inputs, batch_labels)
        for batch_inputs, batch_labels in mini_batches:
            batch_inputs = torch.tensor(batch_inputs, dtype=torch.float32).to(device)
            batch_labels = torch.tensor(batch_labels, dtype=torch.float32).to(device)

            optimizer.zero_grad()
            outputs = model(batch_inputs)
            loss = criterion(outputs, batch_labels.unsqueeze(1))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            batch_count += 1

            #In ra mỗi 50 batch
            if batch_count % 50 == 0:
                print(f"Epoch [{epoch + 1}/{epochs}], Batch [{batch_count}/{len(mini_batches)}], Loss: {loss.item():.4f}")

        epoch_loss = running_loss / len(mini_batches)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {epoch_loss:.4f}")

        save_path = os.path.join(save_dir, f"epoch_{epoch+1}.pt")
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_loss,
        }
        torch.save(checkpoint, save_path)
        print(f"Model checkpoint saved at {save_path}")

### Evaluation metrics

In [218]:
def top_k_accuracy(predictions, labels, k_values=[1, 5, 10, 15]):
    """
    Tính độ chính xác Top-k.

    Với mỗi giá trị k, kiểm tra xem nhãn đúng có nằm trong top-k dự đoán hay không.

    Tham số:
        predictions (Tensor): Tensor có shape (batch_size, num_classes), chứa điểm số dự đoán.
        labels (Tensor): Tensor có shape (batch_size,), chứa chỉ số của lớp đúng.
        k_values (List[int]): Danh sách các giá trị k để tính Top-k Accuracy.

    Trả về:
        dict: Dictionary chứa Top-k Accuracy cho từng giá trị k.
    """
    top_k_accuracies = {}

    # Kiểm tra kích thước của predictions và thay đổi dim nếu cần
    if predictions.dim() == 1 and predictions.size(0) == 1:
        predictions = predictions.unsqueeze(0)  # Thêm một chiều cho predictions nếu nó chỉ có một phần tử
    num_classes = predictions.shape[1]

    for k in k_values:
        actual_k = min(k, num_classes)
        # Lấy top-k chỉ số dự đoán
        _, top_k_indices = torch.topk(predictions, actual_k, largest=True, sorted=False)

        # Kiểm tra xem nhãn đúng có nằm trong top-k không
        top_k_correct = torch.sum((top_k_indices == labels.view(-1, 1)).any(dim=1))
        top_k_accuracy = top_k_correct.item() / len(labels)

        top_k_accuracies[k] = top_k_accuracy

    return top_k_accuracies


def mean_reciprocal_rank(predictions, labels):
    """
    Tính Mean Reciprocal Rank (MRR).

    Với mỗi mẫu, tìm vị trí (rank) của nhãn đúng trong danh sách dự đoán và lấy nghịch đảo (1/rank).

    Tham số:
        predictions (Tensor): Tensor có shape (batch_size, num_classes), chứa điểm số dự đoán.
        labels (Tensor): Tensor có shape (batch_size,), chứa chỉ số của lớp đúng.

    Trả về:
        float: Giá trị Mean Reciprocal Rank.
    """

    if predictions.dim() == 1 and predictions.size(0) == 1:
        predictions = predictions.unsqueeze(0)  # Thêm một chiều cho predictions nếu nó chỉ có một phần tử

    # Lấy top indices (chỉ số các lớp dự đoán từ cao xuống thấp)
    _, top_indices = torch.topk(predictions, predictions.shape[1], largest=True, sorted=False)

    mrr = 0.0

    # Duyệt qua từng mẫu trong batch
    for i in range(len(labels)):
        correct_idx = labels[i].item()  # Lấy chỉ số lớp đúng của mẫu i

        # Tìm vị trí (thứ hạng) của lớp đúng trong top_indices
        rank = (top_indices[i] == correct_idx).nonzero(as_tuple=True)

        if len(rank[0]) > 0:
            rank = rank[0].item() + 1  # Thứ hạng bắt đầu từ 1
            mrr += 1.0 / rank
        else:
            mrr += 0.0  # Nếu lớp đúng không có trong top indices, MRR cho mẫu này là 0

    mrr = mrr / len(labels)  # Tính trung bình MRR cho cả batch
    return mrr


def mean_average_precision(predictions, labels, k_values=[1, 5, 10, 15]):
    """
    Tính Mean Average Precision (MAP) tại nhiều mức top-k.

    Với mỗi k, tính trung bình Precision tại vị trí chứa nhãn đúng (nếu có) trong top-k dự đoán.

    Tham số:
        predictions (Tensor): Tensor có shape (batch_size, num_classes), chứa điểm số dự đoán.
        labels (Tensor): Tensor có shape (batch_size,), chứa chỉ số của lớp đúng.
        k_values (List[int]): Danh sách các giá trị k để tính MAP@k.

    Trả về:
        float: Giá trị trung bình của MAP@k trên tất cả các giá trị k.
    """
    # Nếu predictions chỉ có một chiều, thêm một chiều để có dạng (batch_size, num_classes)
    if predictions.dim() == 1:
        predictions = predictions.unsqueeze(0)  # Thêm một chiều cho predictions nếu chỉ có một phần tử

    map_score = 0.0

    # Giới hạn k không vượt quá số lớp trong predictions
    max_k = min(max(k_values), predictions.size(1))

    _, top_k_indices = torch.topk(predictions, max_k, largest=True, sorted=False)

    for k in k_values:
        avg_precision = 0.0
        for i in range(len(labels)):
            relevant_files = (top_k_indices[i, :k] == labels[i]).sum().item()
            precision_at_k = relevant_files / k
            avg_precision += precision_at_k

        map_score += avg_precision / len(labels)

    map_score = map_score / len(k_values)
    return map_score

### Đánh giá mô hình

In [219]:
def evaluate_model(model, data_test, criterion, device, k_values=[1, 5, 10, 15]):
  model.eval()
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  total_loss = 0.0
  total_samples = 0
  top_k_accuracies = {k: 0 for k in k_values}
  mrr_score = 0.0
  map_score = 0.0

  all_preds =[]
  all_labels = []
  batch_results = []

  with torch.no_grad():
    for X_test, y_test in data_test:
      X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
      y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

      if y_test.dim() == 0:
            y_test = y_test.unsqueeze(0)  # Thêm một chiều để có dạng (1,)

      outputs = model(X_test)

      # Đảm bảo outputs có cùng kích thước với y_test
      #print(f"outputs.shape: {outputs.shape}, y_test.shape: {y_test.shape}")


      loss = criterion(outputs, y_test)
      total_loss += loss.item() * y_test.size(0)
      total_samples += y_test.size(0)


      top_k_acc = top_k_accuracy(outputs, y_test)
      for k in k_values:
        top_k_accuracies[k] += top_k_acc[k]  * y_test.size(0)

      mrr_score += mean_reciprocal_rank(outputs, y_test) * y_test.size(0)
      map_score += mean_average_precision(outputs, y_test) * y_test.size(0)

      all_preds.append(outputs)
      all_labels.append(y_test)

    # Gộp thành tensor duy nhất
    all_preds = torch.cat(all_preds, dim=0)
    all_labels = torch.cat(all_labels, dim=0)

    # Trung bình các chỉ số cho tất cả các batch
    avg_loss = total_loss / total_samples
    avg_top_k_accuracy = {k: top_k_accuracies[k] / total_samples for k in k_values}
    avg_mrr = mrr_score / total_samples
    avg_map = map_score / total_samples

        # Kết quả tổng hợp
    results = {
            'loss': avg_loss,
            'top_k_accuracy': avg_top_k_accuracy,
            'mrr': avg_mrr,
            'map': avg_map
    }
    return results

In [70]:
model = BugLocalization(input_dim=5)
criterion = FocalLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Lấy 1 phần để thử pipeline

In [169]:
print(len(data_train), len(data_test)) # Fold 0 train, fold 1 test

99523 929808


In [171]:
# lấy 1 mẫu nhỏ của fold 0 để train và test thử pipeline
data_train_small = stratified(data_train, 1000)
data_test_small = stratified(data_test, 9000)

data_train_small = create_data(data_train_small, glove_model)
data_test_small = create_data(data_test_small, glove_model)

Creating data for 1000 samples
Creating data for 9000 samples


In [182]:
mini_batches = create_minibatch(data_train_small, batch_size)

#Training
print("---------------------------------------------\n")
print(f"TRAINING")
train_model(model, mini_batches, criterion, optimizer, device, 'temp', epochs)

Creating minibatch for 1000 samples
---------------------------------------------

TRAINING
Epoch [1/30], Loss: 0.0306
Model checkpoint saved at drive/MyDrive/Colab Notebooks/BugLocalization/models/temp_training_model/epoch_1.pt
Epoch [2/30], Loss: 0.0295
Model checkpoint saved at drive/MyDrive/Colab Notebooks/BugLocalization/models/temp_training_model/epoch_2.pt
Epoch [3/30], Loss: 0.0289
Model checkpoint saved at drive/MyDrive/Colab Notebooks/BugLocalization/models/temp_training_model/epoch_3.pt
Epoch [4/30], Loss: 0.0288
Model checkpoint saved at drive/MyDrive/Colab Notebooks/BugLocalization/models/temp_training_model/epoch_4.pt
Epoch [5/30], Loss: 0.0297
Model checkpoint saved at drive/MyDrive/Colab Notebooks/BugLocalization/models/temp_training_model/epoch_5.pt
Epoch [6/30], Loss: 0.0283
Model checkpoint saved at drive/MyDrive/Colab Notebooks/BugLocalization/models/temp_training_model/epoch_6.pt
Epoch [7/30], Loss: 0.0289
Model checkpoint saved at drive/MyDrive/Colab Notebooks/Bug

In [220]:
#Testing
print("---------------------------------------------\n")
print(f"TESTING")
results = evaluate_model(model, data_test_small, criterion, device) #Lỗi hàm top -k, evaluate_model --> in ra key nào cx giống nhau --> check lại evaluation metrics và evaluate_model
print(results)

---------------------------------------------

TESTING
{'loss': 0.02041959121880225, 'top_k_accuracy': {1: 0.9997777777777778, 5: 0.9997777777777778, 10: 0.9997777777777778, 15: 0.9997777777777778}, 'mrr': 0.9997777777777778, 'map': 0.3415907407407639}


### Cross-validation cho project

In [73]:
def cross_validation(project_name, k_fold, model, criterion, optimizer, device, batch_size, epochs):
  for i in range(k_fold - 1):
    data_train, data_test = load_fold_data(project_name, i)
    data_train = stratified(data_train, 50000)
    #data_test = stratified(data_test, 10000)
    # Tạo data sau khi đã xử lý features
    data_train = create_data(data_train, glove_model) #data_train [(vector, label),....]
    data_test = create_data(data_test, glove_model) #[(vector, label),....]
    mini_batches = create_minibatch(data_train, batch_size) # 1 list các batch

    #Training
    print("---------------------------------------------\n")
    print(f"TRAINING ON FOLD {i}")
    train_model(model, mini_batches, criterion, optimizer, device, project_name, epochs)
    #Testing
    print("---------------------------------------------\n")
    print(f"TESTING ON FOLD {i+1}")
    results = evaluate_model(model, data_test, criterion, device)
    print(f"Fold{i} -: {results}")

In [74]:
cross_validation('aspectj', 3, model, criterion, optimizer, device, batch_size, epochs)

Creating data for 50000 samples


KeyboardInterrupt: 